# Imports

In [161]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import tqdm
import matplotlib.pyplot as plt
import time
import os
import copy


# Set device

In [123]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training dataset

In [124]:
data_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

In [168]:
train_dir='./dataset/GTSRB/train/original'

train_dataset = datasets.ImageFolder(train_dir,data_transform)

train_loader = DataLoader(train_dataset, batch_size=200,shuffle=True, num_workers=0)

n_total_steps = len(train_loader)
train_size = len(train_dataset)
class_names = train_dataset.classes

print(class_names)

['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Test dataset

In [169]:
test_dir='./dataset/GTSRB/test'

test_dataset = datasets.ImageFolder(test_dir,data_transform)

test_loader = DataLoader(test_dataset,shuffle=True, num_workers=0)

test_size = len(test_dataset)
class_names = test_dataset.classes

print(class_names)

['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Defining the training

In [170]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
    
            # forward pass
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % 20 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    
            # statistics
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            #to update weights
            scheduler.step()
    
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
    
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            'train', epoch_loss, epoch_acc))
    
    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return model

# Defining the model using ResNet18 as backbone

In [127]:
#### ConvNet as fixed feature extractor ####
# freeze all the network parameters except the final layer settings, to not compute the gradients backward
model_conv = torchvision.models.resnet18(weights=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 43)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as opposed to before.
optimizer_conv = torch.optim.Adam(model_conv.fc.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                        exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
Epoch [1/20], Step [20/197], Loss: 2.5146
Epoch [1/20], Step [40/197], Loss: 2.3447
Epoch [1/20], Step [60/197], Loss: 2.4745
Epoch [1/20], Step [80/197], Loss: 2.5187
Epoch [1/20], Step [100/197], Loss: 2.5092
Epoch [1/20], Step [120/197], Loss: 2.1257
Epoch [1/20], Step [140/197], Loss: 2.5073
Epoch [1/20], Step [160/197], Loss: 2.3934
Epoch [1/20], Step [180/197], Loss: 2.2725
train Loss: 2.5088 Acc: 0.3567
Epoch 1/19
----------
Epoch [2/20], Step [20/197], Loss: 2.4934
Epoch [2/20], Step [40/197], Loss: 2.3936
Epoch [2/20], Step [60/197], Loss: 2.2139
Epoch [2/20], Step [80/197], Loss: 2.4036
Epoch [2/20], Step [100/197], Loss: 2.1705
Epoch [2/20], Step [120/197], Loss: 2.1302
Epoch [2/20], Step [140/197], Loss: 2.1465
Epoch [2/20], Step [160/197], Loss: 2.4035
Epoch [2/20], Step [180/197], Loss: 2.5734
train Loss: 2.4159 Acc: 0.3720
Epoch 2/19
----------
Epoch [3/20], Step [20/197], Loss: 2.6098
Epoch [3/20], Step [40/197], Loss: 2.6668
Epoch [3/20], Step [60

# Save the trained model

In [172]:
pass
print('Finished Training')
os.makedirs('./models', exist_ok=True)
PATH = './models/trained_model.pth'
torch.save(model_conv, PATH)

Finished Training


# Load the model

In [141]:
trained_model=torch.load('./models/trained_model.pth',map_location=device)

# Evaluate the model

In [166]:
def test_model(model, dataloader, dataset_size):
    model.eval()  # Set model to evaluation mode
    
    running_corrects = 0

    with torch.no_grad():  # Disable gradient calculation
        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
    
            # Statistics
            running_corrects += torch.sum(preds == labels.data)
            if (i+1) % 100 == 0:
                print ( f'Evaluating: [{i+1}/{test_size}]')

    # Calculate accuracy
    test_acc = running_corrects.double() / dataset_size

    print('Test Acc: {:.4f}'.format( test_acc))
    

In [167]:
test_model(trained_model, test_loader, test_size)

Evaluating: [100/12630]
Evaluating: [200/12630]
Evaluating: [300/12630]
Evaluating: [400/12630]
Evaluating: [500/12630]
Evaluating: [600/12630]
Evaluating: [700/12630]
Evaluating: [800/12630]
Evaluating: [900/12630]
Evaluating: [1000/12630]
Evaluating: [1100/12630]
Evaluating: [1200/12630]
Evaluating: [1300/12630]
Evaluating: [1400/12630]
Evaluating: [1500/12630]
Evaluating: [1600/12630]
Evaluating: [1700/12630]
Evaluating: [1800/12630]
Evaluating: [1900/12630]
Evaluating: [2000/12630]
Evaluating: [2100/12630]
Evaluating: [2200/12630]
Evaluating: [2300/12630]
Evaluating: [2400/12630]
Evaluating: [2500/12630]
Evaluating: [2600/12630]
Evaluating: [2700/12630]
Evaluating: [2800/12630]
Evaluating: [2900/12630]
Evaluating: [3000/12630]
Evaluating: [3100/12630]
Evaluating: [3200/12630]
Evaluating: [3300/12630]
Evaluating: [3400/12630]
Evaluating: [3500/12630]
Evaluating: [3600/12630]
Evaluating: [3700/12630]
Evaluating: [3800/12630]
Evaluating: [3900/12630]
Evaluating: [4000/12630]
Evaluatin